In [1]:
import torch
import numpy as np
import pandas as pd
# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
if torch.cuda.is_available():
    device = torch.cuda.current_device()
else:
    device =  -1
print(device)

0


In [2]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="sentence-transformers/all-mpnet-base-v2", device=device)

Some weights of the model checkpoint at sentence-transformers/all-mpnet-base-v2 were not used when initializing MPNetForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [3]:
import pandas as pd
labels = open('data/classes.txt').read().splitlines()
all_df = pd.read_csv("data/belief_benchmark.csv")
all_df

,text,sentiment,label
0,Also use of chemicals and machinery on their p...,"NEGATIVE, NEGATIVE",1
1,Sources believe that twenty-five to 30 percent...,UNDETERMINED,0
2,Although these government-led initiatives are ...,UNDETERMINED,0
3,"In other projects of this kind, it is not alwa...",POSITIVE,2
4,Many African policy makers evidently believe t...,POSITIVE,2
...,...,...,...
358,Similar initiatives have and can be initiated ...,POSITIVE,2
359,Collective sales account for an average of 64 ...,UNDETERMINED,0
360,Individual sales account for slightly lower vo...,"UNDETERMINED, UNDETERMINED, UNDETERMINED",0
361,Modernization of the rice value chain The VC t...,UNDETERMINED,0


In [5]:
from datasets import Dataset

dataset = Dataset.from_pandas(all_df)
#dataset.save_to_disk("data/belief_dataset")
dataset

Dataset({
    features: ['text', 'sentiment', 'label'],
    num_rows: 363
})

In [6]:
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

labels = ["neither", "negative", "positive", "both"]
pred_i_list = []
for pred in tqdm(classifier(KeyDataset(dataset, "text"), batch_size=2, candidate_labels=labels)):
    pred_i = labels.index(pred["labels"][0])
    pred_i_list.append(pred_i)

  0%|          | 0/182 [00:00<?, ?it/s]

c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\torch\utils\data\dataloader.py:529: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x00000159D025C6A0> was reported to be 363 (when accessing len(dataloader)), but 364 samples have been fetched. 
  warnings.warn(warn_msg)
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\torch\utils\data\dataloader.py:529: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x00000159D025C6A0> was reported to be 363 (when accessing len(dataloader)), but 365 samples have been fetched. 
  warnings.warn(warn_msg)
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\torch\utils\data\dataloader.py:529: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x00000159D025C6A0> was reported to be 363 (when accessing len(dataloader)), but 366 samples have been fetched. 
  warnings.warn(warn_msg

In [7]:
from sklearn.metrics import classification_report

y_true = all_df["label"]
y_pred = pred_i_list
print(classification_report(y_true, y_pred, target_names=labels))

              precision    recall  f1-score   support

     neither       0.42      0.04      0.07       122
    negative       0.18      0.31      0.23        65
    positive       0.46      0.47      0.46       147
        both       0.09      0.28      0.13        29

    accuracy                           0.28       363
   macro avg       0.29      0.27      0.23       363
weighted avg       0.37      0.28      0.26       363



In [8]:
import gc
with torch.no_grad():
    torch.cuda.empty_cache()
gc.collect()

55

: 